In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import requests
from kakao_map import lat_lon
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("../data/Original_data/train.csv")
test = pd.read_csv("../data/Original_data/test.csv")

print(f"Train Data: {train.shape}")
print(f"Test Data: {test.shape}")

Train Data: (1118822, 52)
Test Data: (9272, 51)


In [3]:
target = train['target']
train = train.drop(columns=['target'])

train['label'] = 'Train' # train
test['label'] = 'Test' # tets

df = pd.concat([train, test], ignore_index=True)
print(f"df: {df.shape}")
df.head()

df: (1128094, 52)


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,label
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train


In [4]:
df.rename(columns={'좌표X':'경도', '좌표Y':'위도'}, inplace=True) 
df['구'] = df['시군구'].str.split().str[1]
df['동'] = df['시군구'].str.split().str[2]
df['도로명_주소'] = df['시군구'] + ' ' + df['도로명']
df['도로명_주소_보완'] = df['시군구'] + ' ' + df['번지'] + ' ' + df['아파트명'] + ' ' + '아파트'
df['지번_주소'] = df['시군구'] + ' ' + df['번지']
front_cols = ['도로명_주소', '도로명_주소_보완', '지번_주소', '구', '동']
other_cols = [col for col in df.columns if col not in front_cols]
df = df[front_cols + other_cols]
print(f"{df[['도로명_주소', '도로명_주소_보완', '구', '동']].isna().sum()}")
df.head()

도로명_주소          0
도로명_주소_보완    2363
구               0
동               0
dtype: int64


,도로명_주소,도로명_주소_보완,지번_주소,구,동,시군구,번지,본번,부번,아파트명,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,경도,위도,단지신청일,label
0,서울특별시 강남구 개포동 언주로 3,서울특별시 강남구 개포동 658-1 개포6차우성 아파트,서울특별시 강남구 개포동 658-1,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
1,서울특별시 강남구 개포동 언주로 3,서울특별시 강남구 개포동 658-1 개포6차우성 아파트,서울특별시 강남구 개포동 658-1,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
2,서울특별시 강남구 개포동 언주로 3,서울특별시 강남구 개포동 658-1 개포6차우성 아파트,서울특별시 강남구 개포동 658-1,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
3,서울특별시 강남구 개포동 언주로 3,서울특별시 강남구 개포동 658-1 개포6차우성 아파트,서울특별시 강남구 개포동 658-1,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train
4,서울특별시 강남구 개포동 언주로 3,서울특별시 강남구 개포동 658-1 개포6차우성 아파트,서울특별시 강남구 개포동 658-1,강남구,개포동,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,Train


In [5]:
missing_lat_lon = df[
    df[['위도', '경도']].isna().any(axis=1)
][['도로명_주소', '도로명_주소_보완', '지번_주소', '구', '동', '번지', '아파트명', '위도', '경도']].drop_duplicates(subset=['도로명_주소', '도로명_주소_보완','지번_주소', '구', '동', '번지', '아파트명'])
missing_lat_lon

,도로명_주소,도로명_주소_보완,지번_주소,구,동,번지,아파트명,위도,경도
975,서울특별시 강남구 개포동 삼성로 14,서울특별시 강남구 개포동 189 개포주공4단지 아파트,서울특별시 강남구 개포동 189,강남구,개포동,189,개포주공4단지,NaN,NaN
986,서울특별시 강남구 개포동 개포로 516,서울특별시 강남구 개포동 185 개포주공7단지 아파트,서울특별시 강남구 개포동 185,강남구,개포동,185,개포주공7단지,NaN,NaN
1031,서울특별시 강남구 개포동 언주로 105,서울특별시 강남구 개포동 655-1 개포현대200동 아파트,서울특별시 강남구 개포동 655-1,강남구,개포동,655-1,개포현대200동,NaN,NaN
1034,서울특별시 강남구 개포동 언주로 110,서울특별시 강남구 개포동 649 경남1 아파트,서울특별시 강남구 개포동 649,강남구,개포동,649,경남1,NaN,NaN
1047,서울특별시 강남구 개포동 언주로 110,서울특별시 강남구 개포동 649 경남2차 아파트,서울특별시 강남구 개포동 649,강남구,개포동,649,경남2차,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1123403,서울특별시 동작구 흑석동 서달로 90,서울특별시 동작구 흑석동 253-89 흑석리버파크자이 아파트,서울특별시 동작구 흑석동 253-89,동작구,흑석동,253-89,흑석리버파크자이,NaN,NaN
1123560,서울특별시 마포구 신수동 토정로17길 6,서울특별시 마포구 신수동 400-1 신수동르끌레브 아파트,서울특별시 마포구 신수동 400-1,마포구,신수동,400-1,신수동르끌레브,NaN,NaN
1124723,서울특별시 서초구 잠원동 잠원로14길 42,서울특별시 서초구 잠원동 52-2 르엘신반포파크애비뉴 아파트,서울특별시 서초구 잠원동 52-2,서초구,잠원동,52-2,르엘신반포파크애비뉴,NaN,NaN
1124909,서울특별시 성동구 송정동 송정길 32,서울특별시 성동구 송정동 67-5 히페리온리버팰리스 아파트,서울특별시 성동구 송정동 67-5,성동구,송정동,67-5,히페리온리버팰리스,NaN,NaN


In [ ]:
def lat_lon_naver(query, client_id, client_secret):
    url = "https://openapi.naver.com/v1/search/local.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret}
    params = {
        "query": query,
        "display": 1}

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"[{response.status_code}] 실패: {query}")
            return None, None

        items = response.json().get('items')
        if not items:
            print(f"위경도 없음: {query}")
            return None, None

        item = items[0]
        lon = float(item['mapx']) / 1e7  # 소수점 맞추기
        lat = float(item['mapy']) / 1e7
        return lat, lon

    except Exception as e:
        print(f"예외 발생 - {query}: {e}")
        return None, None

In [ ]:
client_id = " "
client_secret = " "

for i in range(len(missing_lat_lon)):
    row = missing_lat_lon.iloc[i]

    # 1순위: 도로명 주소
    query = row['도로명_주소']
    lat, lon = lat_lon_naver(query, client_id, client_secret)

    # 2순위: 도로명_주소_보완
    if lat is None or lon is None:
        query = row['도로명_주소_보완']
        lat, lon = lat_lon_naver(query, client_id, client_secret)

    # 3순위: 지번 주소
    if lat is None or lon is None:
        query = row['지번_주소']
        lat, lon = lat_lon_naver(query, client_id, client_secret)

    # 저장
    missing_lat_lon.at[missing_lat_lon.index[i], '위도'] = lat
    missing_lat_lon.at[missing_lat_lon.index[i], '경도'] = lon

    print(f"[{i:04}] {row['아파트명']} | {query} → 위도: {lat}, 경도: {lon}")
    time.sleep(0.2)  # 초당 5건 제한 안전 범위

missing_lat_lon

[0000] 개포주공4단지 | 서울특별시 강남구 개포동 삼성로 14 → 위도: 37.4876162, 경도: 127.0734859
[0001] 개포주공7단지 | 서울특별시 강남구 개포동 개포로 516 → 위도: 37.4882543, 경도: 127.0713323
[0002] 개포현대200동 | 서울특별시 강남구 개포동 언주로 105 → 위도: 37.4816632, 경도: 127.0528962
[0003] 경남1 | 서울특별시 강남구 개포동 언주로 110 → 위도: 37.4848777, 경도: 127.0542279
[0004] 경남2차 | 서울특별시 강남구 개포동 언주로 110 → 위도: 37.4848777, 경도: 127.0542279
[0005] 뉴현대파크빌 | 서울특별시 강남구 개포동 논현로2길 61-4 → 위도: 37.4747929, 경도: 127.0547625
[0006] 리치타운 | 서울특별시 강남구 개포동 개포로28길 28 → 위도: 37.4784855, 경도: 127.0493887
[0007] 삼익대청 | 서울특별시 강남구 개포동 개포로109길 21 → 위도: 37.4950667, 경도: 127.0771093
위경도 없음: 서울특별시 강남구 개포동 논현로2길 36
[0008] 새롬 | 서울특별시 강남구 개포동 1164-13 새롬 아파트 → 위도: 37.473199, 경도: 127.0529597
[0009] 성원대치2단지 | 서울특별시 강남구 개포동 개포로109길 9 → 위도: 37.4946695, 경도: 127.0755079
[0010] 시영 | 서울특별시 강남구 개포동 언주로 21 → 위도: 37.4784871, 경도: 127.0541731
[0011] 아람손프라자/비바빌 | 서울특별시 강남구 개포동 논현로2길 22 → 위도: 37.4727618, 경도: 127.0523151
[0012] 에버그린 | 서울특별시 강남구 개포동 선릉로8길 5 → 위도: 37.4830291, 경도: 127.0615021
[0013] 우성8 | 서울특별시 강남구 개포동 개

,도로명_주소,도로명_주소_보완,지번_주소,구,동,번지,아파트명,위도,경도
975,서울특별시 강남구 개포동 삼성로 14,서울특별시 강남구 개포동 189 개포주공4단지 아파트,서울특별시 강남구 개포동 189,강남구,개포동,189,개포주공4단지,37.487616,127.073486
986,서울특별시 강남구 개포동 개포로 516,서울특별시 강남구 개포동 185 개포주공7단지 아파트,서울특별시 강남구 개포동 185,강남구,개포동,185,개포주공7단지,37.488254,127.071332
1031,서울특별시 강남구 개포동 언주로 105,서울특별시 강남구 개포동 655-1 개포현대200동 아파트,서울특별시 강남구 개포동 655-1,강남구,개포동,655-1,개포현대200동,37.481663,127.052896
1034,서울특별시 강남구 개포동 언주로 110,서울특별시 강남구 개포동 649 경남1 아파트,서울특별시 강남구 개포동 649,강남구,개포동,649,경남1,37.484878,127.054228
1047,서울특별시 강남구 개포동 언주로 110,서울특별시 강남구 개포동 649 경남2차 아파트,서울특별시 강남구 개포동 649,강남구,개포동,649,경남2차,37.484878,127.054228
...,...,...,...,...,...,...,...,...,...
1123403,서울특별시 동작구 흑석동 서달로 90,서울특별시 동작구 흑석동 253-89 흑석리버파크자이 아파트,서울특별시 동작구 흑석동 253-89,동작구,흑석동,253-89,흑석리버파크자이,37.500611,126.960772
1123560,서울특별시 마포구 신수동 토정로17길 6,서울특별시 마포구 신수동 400-1 신수동르끌레브 아파트,서울특별시 마포구 신수동 400-1,마포구,신수동,400-1,신수동르끌레브,37.544880,126.935882
1124723,서울특별시 서초구 잠원동 잠원로14길 42,서울특별시 서초구 잠원동 52-2 르엘신반포파크애비뉴 아파트,서울특별시 서초구 잠원동 52-2,서초구,잠원동,52-2,르엘신반포파크애비뉴,37.516997,127.013692
1124909,서울특별시 성동구 송정동 송정길 32,서울특별시 성동구 송정동 67-5 히페리온리버팰리스 아파트,서울특별시 성동구 송정동 67-5,성동구,송정동,67-5,히페리온리버팰리스,NaN,NaN


In [9]:
# 확인
missing_lat_lon[missing_lat_lon[['위도', '경도']].isna().any(axis=1)].drop_duplicates(subset=['아파트명'])

,도로명_주소,도로명_주소_보완,지번_주소,구,동,번지,아파트명,위도,경도
2030,서울특별시 강남구 도곡동 남부순환로377길 18,서울특별시 강남구 도곡동 183-7 도곡현대힐스테이트102동 아파트,서울특별시 강남구 도곡동 183-7,강남구,도곡동,183-7,도곡현대힐스테이트102동,NaN,NaN
4864,서울특별시 강남구 역삼동 언주로70길 18-8,서울특별시 강남구 역삼동 760-7 현대트라움 아파트,서울특별시 강남구 역삼동 760-7,강남구,역삼동,760-7,현대트라움,NaN,NaN
6084,서울특별시 강동구 길동 양재대로122가길 30,서울특별시 강동구 길동 334-3 시가오? 아파트,서울특별시 강동구 길동 334-3,강동구,길동,334-3,시가오?,NaN,NaN
7671,서울특별시 강동구 명일동 고덕로38길 34-8,서울특별시 강동구 명일동 312-257 우성102동 아파트,서울특별시 강동구 명일동 312-257,강동구,명일동,312-257,우성102동,NaN,NaN
8531,서울특별시 강동구 암사동 상암로23길 28,서울특별시 강동구 암사동 488-3 다성이즈빌Ⅰ 아파트,서울특별시 강동구 암사동 488-3,강동구,암사동,488-3,다성이즈빌Ⅰ,NaN,NaN
...,...,...,...,...,...,...,...,...,...
754757,서울특별시 은평구 증산동 증산로1길 24,서울특별시 은평구 증산동 216-10 LG그린 아파트,서울특별시 은평구 증산동 216-10,은평구,증산동,216-10,LG그린,NaN,NaN
942799,서울특별시 마포구 현석동 토정로18길 41-1,서울특별시 마포구 현석동 128-11 HD101동 아파트,서울특별시 마포구 현석동 128-11,마포구,현석동,128-11,HD101동,NaN,NaN
1023789,서울특별시 구로구 구로동 도림로8길 20,서울특별시 구로구 구로동 799-7 대영B동 아파트,서울특별시 구로구 구로동 799-7,구로구,구로동,799-7,대영B동,NaN,NaN
1087036,서울특별시 구로구 구로동 구로동로12길 4-12,서울특별시 구로구 구로동 807-39 807-39 아파트,서울특별시 구로구 구로동 807-39,구로구,구로동,807-39,807-39,NaN,NaN


In [10]:
df = df.merge(
    missing_lat_lon[['구', '동', '번지', '위도', '경도']],
    on=['구', '동', '번지'],
    how='left',
    suffixes=('', '_보정'))

# 결측치가 있는 값만 보정 좌표로 채우기
mask = df['위도'].isna()
df.loc[mask, '위도'] = df.loc[mask, '위도_보정']
df.loc[mask, '경도'] = df.loc[mask, '경도_보정']
df.drop(columns=['위도_보정', '경도_보정'], inplace=True)

df[['위도', '경도']].isna().sum()

위도    8805
경도    8805
dtype: int64

In [11]:
test_df = df[df['label'] == 'Test']
test_df[test_df[['위도', '경도']].isna().any(axis=1)][['도로명_주소', '구', '동', '번지', '아파트명', '위도', '경도']].drop_duplicates(subset=['아파트명']).reset_index(drop=True)

,도로명_주소,구,동,번지,아파트명,위도,경도
0,서울특별시 강서구 화곡동 까치산로15길 13,강서구,화곡동,26-9,투원씨B동,NaN,NaN
1,서울특별시 관악구 남현동 승방4길 2,관악구,남현동,602-28,네스코수피아,NaN,NaN
2,서울특별시 관악구 봉천동 은천로34길 9,관악구,봉천동,41-259,에메랄드102동,NaN,NaN
3,서울특별시 관악구 봉천동 은천로34길 24,관악구,봉천동,41-265,에메랄드103동,NaN,NaN
4,서울특별시 관악구 봉천동 은천로34길 20,관악구,봉천동,41-274,에메랄드104동,NaN,NaN
5,서울특별시 구로구 구로동 구로동로15길 21-8,구로구,구로동,718-12,부국주택,NaN,NaN
6,서울특별시 금천구 독산동 독산로92길 26,금천구,독산동,992-37,세진웰빙102,NaN,NaN
7,서울특별시 노원구 월계동 월계로42길 97,노원구,월계동,949,꿈의숲SKVIEW,NaN,NaN
8,서울특별시 동작구 흑석동 현충로10길 42,동작구,흑석동,38-58,명수대한양,NaN,NaN
9,서울특별시 서대문구 북아현동 북아현로22길 86,서대문구,북아현동,1-49,우민B,NaN,NaN


In [ ]:
missing_lat_lon['위도'] = None
missing_lat_lon['경도'] = None

# 카카오 API 키
api_key = " "

# 도로명주소 기반 위경도 수집
for i in range(len(missing_lat_lon)):
    row = missing_lat_lon.iloc[i]
    address = row['도로명_주소']

    lat, lon = lat_lon(address, api_key)
    missing_lat_lon.at[missing_lat_lon.index[i], '위도'] = lat
    missing_lat_lon.at[missing_lat_lon.index[i], '경도'] = lon

    print(f"[{i:04}] {row['아파트명']} | 주소: {address} → 위도: {lat}, 경도: {lon}")
    time.sleep(0.2) # 카카오 API는 초당 10건 제한

missing_lat_lon

[0000] 개포주공4단지 | 주소: 서울특별시 강남구 개포동 삼성로 14 → 위도: 37.4861548149371, 경도: 127.068451024326
[0001] 개포주공7단지 | 주소: 서울특별시 강남구 개포동 개포로 516 → 위도: 37.4891406792843, 경도: 127.070871480537
[0002] 개포현대200동 | 주소: 서울특별시 강남구 개포동 언주로 105 → 위도: 37.4816823655211, 경도: 127.052677848982
[0003] 경남1 | 주소: 서울특별시 강남구 개포동 언주로 110 → 위도: 37.4854059409863, 경도: 127.054710026842
[0004] 경남2차 | 주소: 서울특별시 강남구 개포동 언주로 110 → 위도: 37.4854059409863, 경도: 127.054710026842
[0005] 뉴현대파크빌 | 주소: 서울특별시 강남구 개포동 논현로2길 61-4 → 위도: 37.4748147158631, 경도: 127.05480461731
[0006] 리치타운 | 주소: 서울특별시 강남구 개포동 개포로28길 28 → 위도: 37.4773029557679, 경도: 127.050322770307
[0007] 삼익대청 | 주소: 서울특별시 강남구 개포동 개포로109길 21 → 위도: 37.4956264931259, 경도: 127.077191033018
[0008] 새롬 | 주소: 서울특별시 강남구 개포동 논현로2길 36 → 위도: 37.4732789109622, 경도: 127.053101764261
[0009] 성원대치2단지 | 주소: 서울특별시 강남구 개포동 개포로109길 9 → 위도: 37.4957518185981, 경도: 127.075101271591
위경도 없음: 서울특별시 강남구 개포동 언주로 21
[0010] 시영 | 주소: 서울특별시 강남구 개포동 언주로 21 → 위도: None, 경도: None
[0011] 아람손프라자/비바빌 | 주소: 서울특별시 강남구 개포동 논현로2

,도로명_주소,구,동,번지,아파트명,위도,경도
975,서울특별시 강남구 개포동 삼성로 14,강남구,개포동,189,개포주공4단지,37.4861548149371,127.068451024326
986,서울특별시 강남구 개포동 개포로 516,강남구,개포동,185,개포주공7단지,37.4891406792843,127.070871480537
1031,서울특별시 강남구 개포동 언주로 105,강남구,개포동,655-1,개포현대200동,37.4816823655211,127.052677848982
1034,서울특별시 강남구 개포동 언주로 110,강남구,개포동,649,경남1,37.4854059409863,127.054710026842
1047,서울특별시 강남구 개포동 언주로 110,강남구,개포동,649,경남2차,37.4854059409863,127.054710026842
...,...,...,...,...,...,...,...
1123403,서울특별시 동작구 흑석동 서달로 90,동작구,흑석동,253-89,흑석리버파크자이,37.5024629483784,126.963346676611
1123560,서울특별시 마포구 신수동 토정로17길 6,마포구,신수동,400-1,신수동르끌레브,37.5448433292108,126.93592659087
1124723,서울특별시 서초구 잠원동 잠원로14길 42,서초구,잠원동,52-2,르엘신반포파크애비뉴,37.5171626156481,127.013520471373
1124909,서울특별시 성동구 송정동 송정길 32,성동구,송정동,67-5,히페리온리버팰리스,37.5487289822502,127.063395492673


In [ ]:
# 확인
missing_lat_lon[missing_lat_lon[['위도', '경도']].isna().any(axis=1)].drop_duplicates(subset=['아파트명'])

,도로명_주소,구,동,번지,아파트명,위도,경도
1221,서울특별시 강남구 개포동 언주로 21,강남구,개포동,656,시영,None,None
2442,서울특별시 강남구 도곡동 남부순환로 2803,강남구,도곡동,91-5,삼성래미안,None,None
3060,서울특별시 강남구 삼성동 삼성로 649,강남구,삼성동,19-4,상아,None,None
3887,서울특별시 강남구 역삼동 언주로 406,강남구,역삼동,712,개나리4,None,None
4805,서울특별시 강남구 역삼동 언주로70길 24,강남구,역삼동,759-1,은하수,None,None
...,...,...,...,...,...,...,...
754757,서울특별시 은평구 증산동 증산로1길 24,은평구,증산동,216-10,LG그린,None,None
942799,서울특별시 마포구 현석동 토정로18길 41-1,마포구,현석동,128-11,HD101동,None,None
1010678,서울특별시 강동구 성내동 성내로 74,강동구,성내동,547-1,미주,None,None
1070909,서울특별시 용산구 한남동 독서당로34길 22-22,용산구,한남동,1-349,시범,None,None


In [ ]:
df = df.merge(
    missing_lat_lon[['구', '동', '번지', '위도', '경도']],
    on=['구', '동', '번지'],
    how='left',
    suffixes=('', '_보정'))

# 결측치가 있는 값만 보정 좌표로 채우기
mask = df['위도'].isna()
df.loc[mask, '위도'] = df.loc[mask, '위도_보정']
df.loc[mask, '경도'] = df.loc[mask, '경도_보정']
df.drop(columns=['위도_보정', '경도_보정'], inplace=True)

df[['위도', '경도']].isna().sum()

위도    41775
경도    41775
dtype: int64

In [ ]:
test_df = df[df['label'] == 'Test']
test_df[test_df[['위도', '경도']].isna().any(axis=1)][['도로명_주소', '구', '동', '번지', '아파트명', '위도', '경도']].drop_duplicates(subset=['아파트명']).reset_index(drop=True)

,도로명_주소,구,동,번지,아파트명,위도,경도
0,서울특별시 강남구 대치동 선릉로 120,강남구,대치동,503,개포우성1,None,None
1,서울특별시 강남구 대치동 삼성로 150,강남구,대치동,511,한보미도맨션1,None,None
2,서울특별시 강남구 대치동 삼성로 150,강남구,대치동,511,한보미도맨션2,None,None
3,서울특별시 강남구 도곡동 도곡로28길 8,강남구,도곡동,543-7,도곡1차아이파크,None,None
4,서울특별시 강남구 도곡동 남부순환로 2803,강남구,도곡동,91-5,삼성래미안,None,None
5,서울특별시 강남구 도곡동 논현로 205,강남구,도곡동,895-8,한신엠비씨,None,None
6,서울특별시 강남구 역삼동 언주로70길 24,강남구,역삼동,759-1,은하수,None,None
7,서울특별시 강남구 일원동 일원로14길 25,강남구,일원동,719,푸른마을,None,None
8,서울특별시 강동구 암사동 고덕로25길 13-11,강동구,암사동,447-8,대명,None,None
9,서울특별시 강동구 천호동 올림픽로89길 55,강동구,천호동,570,강변그대가갤럭시,None,None
